# ロジスティック回帰を用いた株価分析のデモ

In [1]:
# ライブラリのインポート
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from datetime import date, datetime, timedelta
import talib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression  
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
#warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
# function Klinger Volume Oscillator (KVO)
def klinger_volume_oscillator(df, short_period=34, long_period=55, signal_period=13):
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = np.abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = np.abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df['VolumeForce'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], -df['Volume'])
    df['KVO'] = (df['VolumeForce'].rolling(window=short_period).sum() -
                 df['VolumeForce'].rolling(window=long_period).sum())
    df['KVO_Signal'] = df['KVO'].rolling(window=signal_period).mean()
    return df['KVO'], df['KVO_Signal']

# function Elder's Force Index (EFI)
def elder_force_index(df, period=2):
    df['EFI'] = (df['Close'] - df['Close'].shift(1)) * df['Volume']
    df['EFI_Smoothed'] = df['EFI'].rolling(window=period).mean()
    return df['EFI_Smoothed']

# function true_strength_index (TSI)
def true_strength_index(df, short_period=13, long_period=25):
    df['Change'] = df['Close'] - df['Close'].shift(1)
    df['Smoothed1'] = df['Change'].ewm(span=short_period, adjust=False).mean()
    df['Smoothed2'] = df['Smoothed1'].ewm(span=long_period, adjust=False).mean()
    df['AbsChange'] = abs(df['Change'])
    df['AbsSmoothed1'] = df['AbsChange'].ewm(span=short_period, adjust=False).mean()
    df['AbsSmoothed2'] = df['AbsSmoothed1'].ewm(span=long_period, adjust=False).mean()
    df['TSI'] = (df['Smoothed2'] / df['AbsSmoothed2']) * 100
    return df['TSI']

# オーサムオシレーター
def awesome_oscillator(df, wshort=5, wlong=34):
  # 中値
  df['med'] = (df['High']+df['Low']) / 2
  # オーサムオシレーター
  df['ao'] = df['med'].rolling(wshort).mean() - df['med'].rolling(wlong).mean()
  # 加速減速オシレーター
  df['ado'] = df['ao'] - df['ao'].rolling(wshort).mean()
  return df['ao'], df['ado']

# Choppiness index
def choppiness_index(data, period=14):
    # Calculate the True Range
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift(1))
    low_close = np.abs(data['Low'] - data['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    # Sum of the True Range over the given period
    atr = true_range.rolling(window=period).sum()
    # Calculate the difference between the highest high and lowest low over the period
    max_high = data['High'].rolling(window=period).max()
    min_low = data['Low'].rolling(window=period).min()
    # Calculate the Choppiness Index
    chop = 100 * np.log10(atr / (max_high - min_low)) / np.log10(period)
    return chop

In [3]:
# Download data
ticker = '^N225'
end_date = datetime.today()
start_date = end_date - timedelta(days=730)
data = yf.download(ticker, start_date, end_date)
data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
data.index = pd.to_datetime(data.index)

[*********************100%***********************]  1 of 1 completed


In [4]:
data.tail()

,Close,High,Low,Open,Volume
Date,,,,,
2025-02-03,38520.089844,38948.609375,38401.820312,38932.660156,169200000
2025-02-04,38798.371094,39192.511719,38590.960938,39078.660156,144300000
2025-02-05,38831.480469,39118.949219,38682.179688,38979.210938,151200000
2025-02-06,39066.531250,39190.128906,38851.511719,38977.578125,154700000
2025-02-07,38787.019531,39007.941406,38753.199219,38907.761719,0


In [5]:
# Create features based on prices 
data['SMA50'] = data['Close'].rolling(window=50).mean() 
data['SMA200'] = data['Close'].rolling(window=200).mean() 
data['Volatility'] = data['Close'].rolling(window=20).std() 
data['Daily Return'] = data['Close'].pct_change() 
# 各種指標値の計算
data['SMA_short'] = talib.SMA(data['Close'], timeperiod=5)
data['SMA_long'] = talib.SMA(data['Close'], timeperiod=25)
data['CMO'] = talib.CMO(data['Close'], timeperiod=14)
data['KVO'], data['KVO_Signal'] = klinger_volume_oscillator(data)
data['EFI'] = elder_force_index(data)
data['TSI'] = true_strength_index(data)
data['ARO'] = talib.AROONOSC(data['High'], data['Low'], timeperiod=25)
data['MACD'], data['MACD_signal'], data['MACD_hist'] = talib.MACD(data['Close'], \
        fastperiod=12, slowperiod=26, signalperiod=9)
data['AO'], data['ADO'] = awesome_oscillator(data)
data['TRIX'] = talib.TRIX(data['Close'], timeperiod=9)
data["RSI"] = talib.RSI(data["Close"], timeperiod=14)
data["ADX"] = talib.ADX(data["High"], data["Low"], data["Close"], timeperiod=14)
data["CCI"] = talib.CCI(data["High"], data["Low"], data["Close"], timeperiod=14)
data["slowk"], data["slowd"] = talib.STOCH(data["High"], data["Low"], data["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
data["fastk"], data["fastd"] = talib.STOCHF(data["High"], data["Low"], data["Close"], fastk_period=5, fastd_period=3, fastd_matype=0)
data["WILLR"] = talib.WILLR(data["High"], data["Low"], data["Close"], timeperiod=14)
data["OBV"] = talib.OBV(data["Close"], data["Volume"])
data["upperband"], data["middleband"], data["lowerband"] = talib.BBANDS(data["Close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
data['bol_buy'] = data['lowerband'] - data['Close']
data["SAR"] = talib.SAR(data["High"], data["Low"], acceleration=0, maximum=0)
data['CHOP'] = choppiness_index(data)
data['candle'] = talib.CDLENGULFING(data['Open'],data['High'],data['Low'],data['Close'])

In [6]:
# Remove NaN rows 
data = data.dropna().reset_index()

In [7]:
# Create binary target label for prediction
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)

In [8]:
# Select features and target
features = ['MACD','EFI','RSI','fastd']
#features = ['SMA50','SMA200','Volatility','Daily Return']
X = data[features]
y = data['Target']
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Create and train the model
model = LogisticRegression()  
model.fit(X_train, y_train)

LogisticRegression()

In [10]:
# Evaluate the model
y_pred = model.predict(X_test)  
print("Accuracy:", accuracy_score(y_test, y_pred))  
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.4745762711864407
Confusion Matrix:
 [[16 15]
 [16 12]]


In [11]:
print(X.tail(1).to_numpy())

[[-7.48573244e+01  1.81811779e+10  4.56042768e+01  4.47381666e+01]]


In [12]:
new_data = X.tail(1)
prediction = model.predict(new_data)  
print("Prediction:", prediction)

Prediction: [0]


In [13]:
# Get probability predictions
probabilities = model.predict_proba(X_test)
# Print the first few probability predictions
print(probabilities[-1])

[0.45589404 0.54410596]
